Для подготовки тестового датасета будет использоваться Google Earth Engine.  Это облачная платформа для геопространственного анализа данных в планетарных масштабах.

Импортируем пакет Python API для Google Earth Engine - ee.  

In [1]:
import ee

Проходим аутентификацию. До этого нужно зарегистрироваться на сайте.

In [2]:
ee.Authenticate()

Инициализируем проект, который создали в своем профиле.

In [3]:
ee.Initialize(project='dls-final-project-464106')

In [4]:
import geemap

Был выбран следующий набор аэрофотоснимков: [Brandenburg (Germany) RGBN orthophotos 20 cm](https://developers.google.com/earth-engine/datasets/catalog/Germany_Brandenburg_orthos_20cm?hl=ru#description)

В данном наборе только одно большое изображение. Его можно отобразить на карте:

In [6]:
image = ee.Image('Germany/Brandenburg/orthos/20cm')
m = geemap.Map(center=[52.507899, 13.386091], zoom=19)
m.add_layer(
    image, {}, 'Brandenburg 20cm'
)
display(m)

Map(center=[52.507899, 13.386091], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

Можно посмотреть, что из себя представляет объект image в Google Earth Engine. Можно увидеть тип объекта, id, версию, каналы, информацию по каждому каналу, разрешение и прочее.

In [9]:
image

Само изображение очень огромное (1180000x1230000 px), обрежем его и загрузим на гугл диск.

In [ ]:
# Параметры: 0.2 м/пиксель
scale = 0.2
region_size = scale * 5000  # 1000 м
point = ee.Geometry.Point([13.386091, 52.507899]).transform('EPSG:25833')
# Прямоугольник вокруг точки
region = point.buffer(region_size / 2).bounds()

image = ee.Image('Germany/Brandenburg/orthos/20cm').clip(region)
# Оставляем только каналы RGB
visualized = image.visualize(bands=["R", "G", "B"])
projection = image.projection().getInfo()
# Экспорт
task = ee.batch.Export.image.toDrive(
    image=visualized,
    description='tile_3',
    folder='BrandenburgTiles',
    fileNamePrefix='tile_3',
    crs=projection['crs'],
    crsTransform=projection['transform'],
    region=region.coordinates().getInfo(),
    maxPixels=1e9
)

task.start()
print("Экспорт запущен")

Экспорт запущен


Подключим необходимые библиотеки и подключимся к гугл диску.

In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Посмотрим на сохраненное изображение.

In [ ]:
img_Brandenburg = Image.open('/content/gdrive/MyDrive/BrandenburgTiles/tile_3.tif')
img_Brandenburg

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img_Brandenburg.size

(5097, 5115)

Обрежем немного полученное изображение.

In [ ]:
img_Brandenburg_crop = img_Brandenburg.crop(box=(250,250,3000,3000))
img_Brandenburg_crop

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img_Brandenburg_crop.size

(3000, 3000)

Пропишим пути до директории, где будут храниться изображения.

In [ ]:
TEST_DIR = Path('/content/gdrive/MyDrive/final project dls/dataset/test/')
TEST_DIR

PosixPath('/content/gdrive/MyDrive/final project dls/dataset/test')

In [ ]:
TEST_IMG_DIR = TEST_DIR.joinpath("images_1")
TEST_IMG_DIR.mkdir(exist_ok=True)
TEST_IMG_DIR

PosixPath('/content/gdrive/MyDrive/final project dls/dataset/test/images_1')

In [ ]:
# генератор для разбиения изображения image на подизображения размером sub_img_size
def sub_img_generator(image, sub_img_size):
    for row in range(image.size[1]//sub_img_size[1]):
        for col in range(image.size[0]//sub_img_size[0]):
            yield image.crop(box=(
                    row * sub_img_size[1],
                    col * sub_img_size[0],
                    (row + 1) * sub_img_size[1],
                    (col + 1) * sub_img_size[0]
                )
            )

Разобъем изображение на подизображения 250*250. Такой размер нужен для НС. Сохраним их.

In [ ]:
sub_gen = sub_img_generator(img_Brandenburg_crop, (250,250))
for i, subimg in enumerate(sub_gen):
    subimg.save(f'{TEST_IMG_DIR}/Brandenburg_{i}.png')

Далее изображения были визуально оценены, некоторые были удалены. Всего осталось 58 изображений. Они были размечены в CVAT.